In [3]:
# set cuda visible devices
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

import os
if is_notebook():
    os.environ["CUDA_VISIBLE_DEVICES"] = "1" #"1"
    # os.environ['CUDA_LAUNCH_BLOCKING']="1"
    # os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

import matplotlib 
if not is_notebook():
    matplotlib.use('Agg')

In [4]:
import os
import math
import json
import random as rnd
from typing import Optional, Callable
from tqdm import tqdm
from collections import defaultdict
from functools import partial
from datetime import datetime
from dataclasses import dataclass 
from itertools import product

from omegaconf import OmegaConf
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
import matplotlib.pyplot as plt
import pandas as  pd
import torchvision.utils as vision_utils
from PIL import Image
import torchvision
from torchvision import transforms
from matplotlib.ticker import NullFormatter
from sklearn.decomposition import PCA

from losses.divdis import DivDisLoss 
from losses.divdis import DivDisLoss
from losses.ace import ACELoss
from losses.conf import ConfLoss
from losses.dbat import DBatLoss
from losses.pass_through import PassThroughLoss
from losses.smooth_top_loss import SmoothTopLoss
from losses.loss_types import LossType

from models.backbone import MultiHeadBackbone
from models.multi_model import MultiNetModel, freeze_heads
from models.lenet import LeNet

from spurious_datasets.cifar_mnist import get_cifar_mnist_datasets
from spurious_datasets.fmnist_mnist import get_fmnist_mnist_datasets
from spurious_datasets.toy_grid import get_toy_grid_datasets
from spurious_datasets.waterbirds import get_waterbirds_datasets
from spurious_datasets.cub import get_cub_datasets
from spurious_datasets.camelyon import get_camelyon_datasets
from spurious_datasets.multi_nli import get_multi_nli_datasets
from spurious_datasets.civil_comments import get_civil_comments_datasets
from spurious_datasets.celebA import get_celebA_datasets

from utils.utils import to_device, batch_size, feature_label_ls
from utils.logger import Logger
from utils.act_utils import get_acts_and_labels, plot_activations, compute_probe_acc

/nas/ucb/oliveradk/miniforge3/envs/od_3_10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
mix_rate = 0.5
device = "cuda"


from torchvision import models
from torchvision.models.resnet import ResNet50_Weights
resnet_builder = lambda: models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)    
model_builder = lambda: torch.nn.Sequential(*list(resnet_builder().children())[:-1])
resnet_50_transforms = ResNet50_Weights.IMAGENET1K_V1.transforms()
model_transform = transforms.Compose([
    transforms.Resize(resnet_50_transforms.resize_size * 2, interpolation=resnet_50_transforms.interpolation),
    transforms.CenterCrop(resnet_50_transforms.crop_size),
    transforms.Normalize(mean=resnet_50_transforms.mean, std=resnet_50_transforms.std)
])
feature_dim = 2048
model = model_builder().to(device)

Files already downloaded and verified
Files already downloaded and verified


/nas/ucb/oliveradk/diverse-gen/spurious_datasets/dominos.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(dataset_path)


In [ ]:

source_train, source_val, target_train, target_val, target_test = get_cifar_mnist_datasets(
        target_mix_rate_0_1= mix_rate / 2,
        target_mix_rate_1_0= mix_rate / 2, 
        transform=model_transform, 
        pad_sides=True
    )

In [13]:
batch_size = 64
source_train_loader = DataLoader(source_train, batch_size=batch_size, shuffle=True, num_workers=4)
source_val_loader = DataLoader(source_val, batch_size=batch_size, shuffle=False, num_workers=4)
target_train_loader = DataLoader(target_train, batch_size=batch_size, shuffle=True, num_workers=4)
target_val_loader = DataLoader(target_val, batch_size=batch_size, shuffle=False, num_workers=4)
target_test_loader = DataLoader(target_test, batch_size=batch_size, shuffle=False, num_workers=4)


In [18]:
train_labels.shape, test_labels.shape

(torch.Size([3472, 2]), torch.Size([980, 2]))

In [20]:

train_acts, train_labels = get_acts_and_labels(model, target_train_loader, device)
test_acts, test_labels = get_acts_and_labels(model, target_test_loader, device)

 38%|███▊      | 21/55 [00:03<00:04,  6.86it/s]


KeyboardInterrupt: 

In [21]:
probe_acc, probe_acc_alt = compute_probe_acc(train_acts, train_labels, test_acts, test_labels, classes_per_feat=[2,2])
print(f"Probe accuracy: {probe_acc:.2f}")
print(f"Probe accuracy alt: {probe_acc_alt:.2f}")

/nas/ucb/oliveradk/miniforge3/envs/od_3_10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Probe accuracy: 0.84
Probe accuracy alt: 1.00


/nas/ucb/oliveradk/miniforge3/envs/od_3_10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# TODO: integrate proper train / test split into main codebase, de-risk on one run, then just rerun 
# cifar-mnist (in new directory) and plot probe accuracy over epochs